Load Thermal-FIST library and the common headers

In [1]:
#include "inc/loadFISTlibs.h"

###############################################################################
#                                                                             #
# This is Thermal-FIST version 1.2.2                                          #
#                                                                             #
# Copyright (c) 2019 Volodymyr Vovchenko <vovchenko@fias.uni-frankfurt.de>    #
#                                                                             #
# Distributed under the GNU General Public License 3.0 (GPLv3 or later)       #
#                                                                             #
# Please cite when using this code:                                           #
# V. Vovchenko, H. Stoecker, arXiv:1901.05249 [nucl-th]                       #
#                                                                             #
# The latest version is available at https://github.com/vlvovch/Thermal-FIST  #
#                                       

In [2]:
using namespace std;

In [3]:
using namespace thermalfist;

In [4]:
#include "inc/AuxiliaryFunctions.h"

Parametrization of the chemical freeze-out curve

In [5]:
// If true uses the freeze-out curve from V.~Vovchenko, V.~V.~Begun and M.~I.~Gorenstein, arXiv:1512.08025 [nucl-th]
// Otherwise (default) uses the curve from J.~Cleymans, H.~Oeschler, K.~Redlich and S.~Wheaton, hep-ph/0511094
bool our_frz_curve = false;

In [6]:
double muBss(double ecm) 
{
  if (our_frz_curve)
    return 1.477 / (1. + 0.343 * ecm);
  else
    return 1.308 / (1. + 0.273 * ecm);
}

In [7]:
double Tss(double ecm) 
{
	double tmpmuB = muBss(ecm);
	if (our_frz_curve)
		return 0.157 - 0.087 * tmpmuB * tmpmuB - 0.092 * tmpmuB * tmpmuB * tmpmuB * tmpmuB;
	else
		return 0.166 - 0.139 * tmpmuB * tmpmuB - 0.053 * tmpmuB * tmpmuB * tmpmuB * tmpmuB;
}

Routine which will compute Fig. 1

In [8]:
// Routine which calculates the energy dependence of the GCE densities 
// of hadrons with various conserved charges and outputs them to a file

void DensitiesVsEcm(ThermalModelBase *model, double smin = 3., double smax = 3000., int iterss = 10, 
                    std::string filename = "out/GCEDensitiesVsEcm.dat")
{
    //Output on the screen and to a file
    vector<std::ostream*> outs;
    outs.push_back(&std::cout);
    
    ofstream fout(filename.c_str());
    outs.push_back(&fout);
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "sqrts[GeV]";
        *out << setw(15) << "T[MeV]";
        *out << setw(15) << "muB[MeV]";
        *out << setw(15) << "muQ[MeV]";
        *out << setw(15) << "muS[MeV]";
        *out << setw(15) << "nH[fm-3]";
        *out << setw(15) << "nB/nH";
        *out << setw(15) << "nQ/nH";
        *out << setw(15) << "nS/nH";
        *out << endl;
    }
    

    model->ConstrainMuS(true);
    model->ConstrainMuQ(true);
    model->ConstrainMuC(true);
    model->SetQoverB(0.4);

    double logSmin = log(smin);
    double logSmax = log(smax);
    double dlogS = (logSmax - logSmin) / iterss;
    for (int is = 0; is <= iterss; ++is) {
        double ss = exp(logSmin + is * dlogS);
        double T = Tss(ss);
        double muB = muBss(ss);

        model->SetTemperature(T);
        model->SetBaryonChemicalPotential(muB);
        model->FixParameters();

        model->CalculateDensities();

        double nH = 0., nB = 0., nQ = 0., nS = 0.;
        //nH = model->CalculateHadronDensity();
        for (int i = 0; i < model->Densities().size(); ++i) {
            const ThermalParticle& part = model->TPS()->Particle(i);
            nH += model->Densities()[i];

            if (part.BaryonCharge() != 0)
                nB += model->Densities()[i];

            if (part.ElectricCharge() != 0)
                nQ += model->Densities()[i];

            if (part.Strangeness() != 0)
                nS += model->Densities()[i];
        }
        
        for(std::ostream* out : outs) {
            *out << setw(15) << ss;
            *out << setw(15) << T * 1.e3;
            *out << setw(15) << muB * 1.e3;
            *out << setw(15) << model->Parameters().muQ * 1.e3;
            *out << setw(15) << model->Parameters().muS * 1.e3;
            *out << setw(15) << nH;
            *out << setw(15) << nB / nH;
            *out << setw(15) << nQ / nH;
            *out << setw(15) << nS / nH;
            *out << endl;
        }
    }
}

Routine which will compute temperature dependence at \mu = 0

In [9]:
// Routine which calculates the temperature dependence at \mu = 0 of the GCE densities 
// of hadrons with various conserved charges and outputs them to a file

void DensitiesVsT(ThermalModelBase *model, double Tmin = 0.100, double Tmax = 0.300, double dT = 0.001, 
                  std::string filename = "GCEDensitiesVsT.LHC.dat")
{
    //Output on the screen and to a file
    vector<std::ostream*> outs;
    outs.push_back(&std::cout);

    ofstream fout(filename.c_str());
    outs.push_back(&fout);

    for(std::ostream* out : outs) {
        *out << setw(15) << "T[MeV]";
        *out << setw(15) << "nH[fm-3]";
        *out << setw(15) << "nB/nH";
        *out << setw(15) << "nQ/nH";
        *out << setw(15) << "nS/nH";
        *out << endl;
    }

    model->SetBaryonChemicalPotential(0.);
    model->SetElectricChemicalPotential(0.);
    model->SetStrangenessChemicalPotential(0.);
    model->SetCharmChemicalPotential(0.);

    for (double T = Tmin; T <= Tmax; T += dT) {

        model->SetTemperature(T);
        model->CalculateDensities();

        double nH = 0., nB = 0., nQ = 0., nS = 0.;
        //nH = model->CalculateHadronDensity();
        for (int i = 0; i < model->Densities().size(); ++i) {
            const ThermalParticle& part = model->TPS()->Particle(i);
            nH += model->Densities()[i];

            if (part.BaryonCharge() != 0)
                nB += model->Densities()[i];

            if (part.ElectricCharge() != 0)
                nQ += model->Densities()[i];

            if (part.Strangeness() != 0)
                nS += model->Densities()[i];
        }

        for(std::ostream* out : outs) {
            *out << setw(15) << T * 1.e3;
            *out << setw(15) << nH;
            *out << setw(15) << nB / nH;
            *out << setw(15) << nQ / nH;
            *out << setw(15) << nS / nH;
            *out << endl;
        }
    }
}

Run the calculation of energy dependence of the GCE densities

In [10]:
// PDG2014 particle list with light nuclei
ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list-withnuclei.dat");

In [11]:
// Ideal HRG model
ThermalModelIdeal model(&parts);

In [12]:
// Energy-dependent Breit-Wigner widths
ThermalParticle::ResonanceWidthIntegration useWidth = ThermalParticle::eBW;
model.SetUseWidth(useWidth);

In [13]:
// Quantum statistics
bool useQStats = true;
model.SetStatistics(useQStats);
// Include quantum statistics for mesons only
if (useQStats){
    for (int i = 0; i < model.TPS()->Particles().size(); ++i) {
        ThermalParticle &part = model.TPS()->Particle(i);
        if (part.BaryonCharge() != 0)
            part.UseStatistics(false);
    }
}

In [14]:
// Run the collision energy dependence calculation (Fig. 1)
string filename = "GCEDensitiesVsEcm.dat";
if (model.TPS()->ResonanceWidthIntegrationType() == ThermalParticle::eBW)
    filename = "eBW." + filename;
filename = "out/1906.03145.1_" + filename;
    
DensitiesVsEcm(&model, 2., 5100., 200, filename);

     sqrts[GeV]         T[MeV]       muB[MeV]       muQ[MeV]       muS[MeV]       nH[fm-3]          nB/nH          nQ/nH          nS/nH
              2        39.3465        846.054       -21.2128        152.385      0.0229593       0.976792       0.477906    0.000576077
           2.08        43.5809        834.269       -22.1063        151.366      0.0264619       0.965475       0.483149      0.0013676
        2.16319        47.7595        822.356       -22.6331        150.293      0.0302319       0.952129        0.48973     0.00276088
        2.24972        51.8785        810.322       -22.8278        149.157      0.0342878       0.937042       0.497105     0.00493366
         2.3397        55.9341        798.175       -22.7435        147.957      0.0386433         0.9205       0.504791     0.00802534
        2.43329        59.9229        785.922       -22.4409          146.7      0.0433091       0.902761       0.512413      0.0121239
        2.53062        63.8414        773.572   

        21.0374        160.695        193.973       -5.42812        45.6817       0.464916       0.215354       0.547621       0.293378
        21.8789        161.043        187.583       -5.26016        44.2146       0.468015       0.211981       0.547315        0.29402
         22.754         161.37        181.368       -5.09585        42.7843       0.470935       0.208802       0.547026       0.294618
        23.6641        161.677        175.328       -4.93521        41.3906       0.473685       0.205806       0.546753       0.295174
        24.6106        161.965        169.459       -4.77827        40.0332       0.476273       0.202987       0.546495        0.29569
         25.595        162.234        163.757       -4.62504        38.7119       0.478707       0.200334       0.546253       0.296171
        26.6188        162.487        158.221       -4.47555        37.4262       0.480995        0.19784       0.546024       0.296618
        27.6835        162.724        152.847   

        230.137        165.942        20.4927       -0.59231        4.88833       0.512934       0.162897        0.54279       0.302402
        239.342        165.946        19.7165      -0.569889         4.7032       0.512975       0.162852       0.542786       0.302409
        248.916         165.95        18.9692      -0.548303        4.52499       0.513013        0.16281       0.542782       0.302415
        258.872        165.954        18.2498      -0.527522        4.35342       0.513048       0.162772       0.542779       0.302421
        269.226        165.957        17.5573      -0.507517        4.18827       0.513081       0.162736       0.542775       0.302427
        279.995         165.96        16.8908       -0.48826         4.0293       0.513111       0.162703       0.542772       0.302432
        291.194        165.963        16.2492      -0.469723        3.87629       0.513138       0.162672       0.542769       0.302436
        302.842        165.966        15.6318   

        2517.57        165.999        1.90034     -0.0549467        0.45337        0.51348       0.162297       0.542734       0.302494
        2618.27            166        1.82736     -0.0528364       0.435958        0.51348       0.162297       0.542734       0.302494
           2723            166        1.75717     -0.0508071       0.419213        0.51348       0.162297       0.542734       0.302494
        2831.91            166        1.68968     -0.0488556       0.403111       0.513481       0.162296       0.542734       0.302494
        2945.18            166        1.62477     -0.0469789       0.387627       0.513481       0.162296       0.542734       0.302494
        3062.99            166        1.56236     -0.0451743       0.372736       0.513481       0.162296       0.542734       0.302494
         3185.5            166        1.50234     -0.0434389       0.358417       0.513481       0.162295       0.542734       0.302494
        3312.92            166        1.44462   